<a href="https://colab.research.google.com/github/thxsxth/POMDP_RLSepsis/blob/master/discrete_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd 'drive/My Drive/sepsis3-cohort'

/content/drive/My Drive/sepsis3-cohort


In [ ]:
import torch
import numpy as np
import pandas as pd
import datetime as dt
import random
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import os
import glob
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device='cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('RL_2.csv')
df=df.iloc[:,1:]

## Discretizing Actions

Get rid of the negative Volumes

In [7]:
df['42']=df['42'].apply(lambda x:max(x,0))

First let's try to set up a column indicating if there has been a treatment or not, and then add this to the cuts

In [ ]:
df['Vaso_T/F']=(df['41']!=0.0).astype(int)
df['Fluids_T/F']=(df['42']!=0.0).astype(int)

#### Let's find quartiles of non zero Vaso rates and non zero Fluids and use it to bin the actions

 Find non zero Vaso quartiles, and Fluid quartiles

In [ ]:
df[df['41']!=0]['41'].describe()

count    454505.000000
mean          3.294228
std          12.740657
min           0.000900
25%           0.067500
50%           0.142469
75%           0.408000
max        4287.142112
Name: 41, dtype: float64

Fluids

In [ ]:
df[df['42']!=0]['42'].describe()

count    2.033608e+06
mean     1.771064e+02
std      8.050111e+02
min      6.000000e-03
25%      1.000000e+01
50%      5.000000e+01
75%      2.107170e+02
max      1.000150e+06
Name: 42, dtype: float64

In [ ]:
df['Fluids_T/F'].value_counts(normalize=True)

0    0.581838
1    0.418162
Name: Fluids_T/F, dtype: float64

Let's bin based on quartiles

In [8]:
Vaso_cuts,vaso_bins=pd.cut(df['41'],bins=[-1e-6,0.000900/2,0.067500,0.142469,0.408000,np.inf],labels=False,retbins=True)

In [ ]:
vaso_bins

array([-1.00000e-06,  4.50000e-04,  6.75000e-02,  1.42469e-01,
        4.08000e-01,          inf])

In [9]:
Vaso_cuts.value_counts(normalize=True)

0    0.906542
1    0.023898
4    0.023369
3    0.023357
2    0.022834
Name: 41, dtype: float64

In [10]:
df['Vaso_cuts']=Vaso_cuts.values

In [11]:
Fluids_cuts,fluid_bins=pd.cut(df['42'],bins=[-1e-6,6.000000e-03/2,1.000000e+01,5.000000e+01,2.107170e+02,np.inf],labels=False,retbins=True)

In [ ]:
Fluids_cuts.value_counts(normalize=True)

0    0.581838
1    0.113278
4    0.104540
2    0.102095
3    0.098249
Name: 42, dtype: float64

In [ ]:
fluid_bins

array([-1.00000e-06,  3.00000e-03,  1.00000e+01,  5.00000e+01,
        2.10717e+02,          inf])

In [12]:
df['Fluid_cuts']=Fluids_cuts.values

In [13]:
df['Fluid_cuts'].value_counts(normalize=True),df['Vaso_cuts'].value_counts(normalize=True)

(0    0.581838
 1    0.113278
 4    0.104540
 2    0.102095
 3    0.098249
 Name: Fluid_cuts, dtype: float64, 0    0.906542
 1    0.023898
 4    0.023369
 3    0.023357
 2    0.022834
 Name: Vaso_cuts, dtype: float64)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,Vaso_cuts,Fluid_cuts,action
0,46.0,118.0,62.0,26.0,0.0,94.0,109.0,36.388889,0.0,0.0,0.0,0.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.502859,1.030435,0.437960,65.957603,82.0,0.0,75.1,0.0,0.0,-0.250,0,0,0
1,41.0,111.0,58.0,23.0,2.0,95.0,109.0,36.388889,1.0,0.0,0.0,1.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.482850,0.995847,0.452339,66.413605,82.0,0.0,75.1,0.0,0.0,-0.375,0,0,0
2,44.0,108.0,62.0,19.0,5.0,98.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.521583,0.918802,0.458562,66.626884,82.0,0.0,75.1,0.0,0.0,-0.025,0,0,0
3,41.0,96.0,60.0,20.0,5.0,96.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.565697,0.881036,0.521476,67.943565,82.0,0.0,75.1,0.0,0.0,-0.025,0,0,0
4,46.0,95.0,65.0,20.0,5.0,93.0,124.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.604537,0.859168,0.516133,67.920723,82.0,0.0,75.1,0.0,0.0,-0.025,0,0,0


#### Final 25 element discrete action set:

In [14]:
all_acts=np.arange(25).reshape(5,5)

In [ ]:
all_acts


array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [15]:
df['action']=all_acts[df.Vaso_cuts,df.Fluid_cuts]

In [16]:
df['action'].value_counts(normalize=True)

0     0.551102
1     0.095689
4     0.092879
3     0.085614
2     0.081257
5     0.008484
10    0.008193
15    0.007280
20    0.006778
22    0.006035
6     0.005731
17    0.005174
7     0.004983
12    0.004646
11    0.004206
16    0.004031
21    0.003621
18    0.003531
24    0.003531
23    0.003404
19    0.003341
13    0.003092
14    0.002698
8     0.002609
9     0.002091
Name: action, dtype: float64

In [ ]:
df.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', 'No_vaso', 'No_Fluids',
       'Vaso_T/F', 'Fluids_T/F', 'Vaso_cuts', 'Fluid_cuts', 'action'],
      dtype='object')

### Normalize some columns

In [17]:
temp=df.describe()
means=temp.loc['mean'].values
stds=temp.loc['std'].values

cols=[0,1,2,3,4,5,6,7,11,38,40]

df.iloc[:,cols]=(df.iloc[:,cols]-means[cols])/stds[cols]

In [ ]:
means

array([ 6.10516501e+01,  8.58273762e+01,  7.92426131e+01,  2.04898922e+01,
        5.01451572e+00,  9.69608508e+01,  1.22969488e+02,  3.68584679e+01,
        9.81975564e-01,  1.46565705e+00,  3.55564596e-01,  1.11590816e+00,
       -1.81130535e-01,  1.44406739e-01, -2.60957833e-02,  2.73962882e-02,
       -8.49182310e-02, -1.21395924e-01, -1.95291586e-01,  1.17347470e-01,
       -6.53730051e-02,  4.81292435e-02,  1.83998721e-02, -8.83261583e-03,
        1.66786174e-01,  1.53489572e-01, -4.13890258e-01,  1.44375674e-01,
        9.74360039e-02, -5.18692390e-01, -4.22981059e-02,  2.25997100e-01,
        1.50821006e-01, -1.43958070e-02,  8.30747706e-01,  1.22739589e+00,
        6.55076381e-01,  6.86925131e+01,  6.45461807e+01,  5.79334730e-01,
        8.28079896e+01,  2.87738281e-01,  7.31972110e+01,  2.31048253e-02])

In [ ]:
stds

array([1.43773336e+01, 1.62935055e+01, 1.55328782e+01, 5.68626569e+00,
       3.63318264e+00, 2.98583709e+00, 2.15652101e+01, 7.25869497e-01,
       9.40552482e-01, 1.37994811e+00, 9.15302747e-01, 1.43479946e+00,
       8.83415344e-01, 9.49437179e-01, 9.32259612e-01, 9.25629998e-01,
       7.24986223e-01, 7.92938237e-01, 7.84110025e-01, 1.05471056e+00,
       8.18574593e-01, 8.90501291e-01, 9.66663700e-01, 7.26903595e-01,
       4.20460721e-01, 4.07409584e-01, 3.37655265e-01, 3.63527999e-01,
       3.64973140e-01, 3.48002144e-01, 3.52433500e-01, 3.65747372e-01,
       4.12381825e-01, 3.53033810e-01, 2.01268080e-01, 4.51708516e-01,
       1.33722613e-01, 1.50245920e+00, 1.59252179e+01, 4.93665930e-01,
       2.50102486e+01, 2.64738699e+00, 2.16311372e+02, 9.30323750e-01])

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,No_vaso,No_Fluids,Vaso_T/F,Fluids_T/F,Vaso_cuts,Fluid_cuts
0,-1.046870,1.974396,-1.110013,0.969004,-1.378317,-0.991096,-0.647660,-0.646901,0.0,0.0,0.0,-0.777651,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.502859,1.030435,0.437960,65.957603,1.096018,0.0,-0.308252,0.0,0.0,-0.250,1,1,0,0,0,0
1,-1.394644,1.544799,-1.367532,0.441438,-0.828779,-0.656335,-0.647660,-0.646901,1.0,0.0,0.0,-0.080965,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.482850,0.995847,0.452339,66.413605,1.096018,0.0,-0.308252,0.0,0.0,-0.375,1,1,0,0,0,0
2,-1.185979,1.360686,-1.110013,-0.261983,-0.004472,0.347947,-0.230333,-0.646901,1.0,0.0,0.0,2.009094,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.521583,0.918802,0.458562,66.626884,1.096018,0.0,-0.308252,0.0,0.0,-0.025,1,1,0,0,0,0
3,-1.394644,0.624235,-1.238772,-0.086127,-0.004472,-0.321574,-0.230333,-0.646901,1.0,0.0,0.0,2.009094,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.565697,0.881036,0.521476,67.943565,1.096018,0.0,-0.308252,0.0,0.0,-0.025,1,1,0,0,0,0
4,-1.046870,0.562864,-0.916875,-0.086127,-0.004472,-1.325856,0.047884,-0.646901,1.0,0.0,0.0,2.009094,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.604537,0.859168,0.516133,67.920723,1.096018,0.0,-0.308252,0.0,0.0,-0.025,1,1,0,0,0,0


### Data set class for Discrete RL

In [18]:
class Discrete_RL_dataset(Dataset):
  """
   Need the df Ready and scaled
   Normalization is Done before

  """

  def __init__(self,scale_rewards=False,df=df):
    self.scale_rewards=scale_rewards
    self.df=df

  def __len__(self):
    return df.shape[0]

  def __getitem__(self,idx):
    temp=self.df.iloc[idx,:]
    # Rewards is always +- 15 at the terminal step and 
    done=int(np.abs(temp['43'])==15)
    states=torch.FloatTensor(temp.iloc[:41].values).to(device)
    assert states.shape==(41,)
    if done:
      next_states=torch.zeros_like(states).to(device)
    else:
      next_states=torch.FloatTensor(self.df.iloc[idx+1,:].values[:41]).to(device)

    assert next_states.shape==(41,)

    reward=temp['43']
    if self.scale_rewards:
      reward=(rewards-2.313230e-02)/9.304778e-01
    
    action=int(temp.action)

    # assert actions.shape==(2,)

    return states,next_states,action,reward,done
    

In [19]:
ds=Discrete_RL_dataset()
loader=DataLoader(ds,batch_size=64*2,shuffle=True)

In [ ]:
ds[980]

(tensor([ 6.6008e-02, -9.7141e-01, -4.0184e-01, -8.6127e-02, -8.2878e-01,
          6.8271e-01, -7.8677e-01, -1.3357e+00,  1.0000e+00,  1.0000e+00,
          0.0000e+00, -7.7765e-01, -1.4824e+00,  3.1003e+00, -4.7004e-01,
         -2.1671e+00, -6.9930e-02,  2.7027e-02, -4.2191e-01,  1.3858e+00,
         -8.0786e-01, -6.8430e-01, -3.7220e-01, -9.5795e-02,  7.0819e-01,
          8.4326e-01, -6.2902e-01, -5.6409e-01, -8.9073e-01, -5.4102e-01,
         -2.3203e-01,  5.2767e-01,  6.9346e-01, -2.7490e-01,  9.0433e-01,
          1.5054e+00,  7.7138e-01,  7.0097e+01,  3.4248e-01,  1.0000e+00,
          3.7781e+00]),
 tensor([-1.6729e+00, -1.0942e+00, -1.6894e+00, -7.8955e-01, -8.2878e-01,
          6.8271e-01, -1.0650e+00, -1.3357e+00,  1.0000e+00,  1.0000e+00,
          0.0000e+00, -7.7765e-01, -1.4824e+00,  3.1003e+00, -4.7004e-01,
         -2.1671e+00, -6.9930e-02,  2.7027e-02, -4.2191e-01,  1.3858e+00,
         -8.0786e-01, -6.8430e-01, -3.7220e-01, -9.5795e-02,  7.0819e-01,
          8.43

### BCQ Moduels

In [20]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [21]:
class FC_Q(nn.Module):
	def __init__(self, state_dim, num_actions,q_layers=4,i_layers=4):
		super(FC_Q, self).__init__()
		self.q1 = nn.Linear(state_dim, 512)
		self.q2 = nn.Linear(512, 512)
		self.q3 = nn.Linear(512, num_actions)
    
		self.i1 = nn.Linear(state_dim,512)
		self.i2 = nn.Linear(512, 512)
		self.i3 = nn.Linear(512, num_actions)		


	def forward(self, state):
		q = F.relu(self.q1(state))
		q = F.relu(self.q2(q))

		i = F.relu(self.i1(state))
		i = F.relu(self.i2(i))
		i = F.relu(self.i3(i))
		return self.q3(q), F.log_softmax(i, dim=1), i

In [22]:
class discrete_BCQ(object):
  def __init__(
		self, 
		is_atari,
		num_actions,
		state_dim,
		device,
		BCQ_threshold=0.3,
		discount=0.99,
		optimizer="Adam",
		optimizer_parameters={},
		polyak_target_update=False,
		target_update_frequency=8e3,
		tau=0.005,
		initial_eps = 1,
		end_eps = 0.001,
		eps_decay_period = 25e4,
		eval_eps=0.001,
	):
	
		  self.device = device

		  # Determine network type
		  self.Q = FC_Q(state_dim, num_actions).to(self.device)
		  self.Q_target = copy.deepcopy(self.Q)
		  self.Q_optimizer = getattr(torch.optim, optimizer)(self.Q.parameters(), **optimizer_parameters)

		  self.discount = discount

		  # Target update rule
		  self.maybe_update_target = self.polyak_target_update if polyak_target_update else self.copy_target_update
		  self.target_update_frequency = target_update_frequency
		  self.tau = tau

		  # Decay for eps
		  self.initial_eps = initial_eps
		  self.end_eps = end_eps
		  self.slope = (self.end_eps - self.initial_eps) / eps_decay_period

		  # Evaluation hyper-parameters
		  self.state_shape =(-1, state_dim)
		  self.eval_eps = eval_eps
		  self.num_actions = num_actions

		  # Threshold for "unlikely" actions
		  self.threshold = BCQ_threshold

		  # Number of training iterations
		  self.iterations = 0


  def select_action(self, state, eval=False):
		# Select action according to policy with probability (1-eps)
		# otherwise, select random action
		  if np.random.uniform(0,1) > self.eval_eps:
			  with torch.no_grad():
				  state = torch.FloatTensor(state).reshape(self.state_shape).to(self.device)
				  q, imt, i = self.Q(state)
				  imt = imt.exp()
				  imt = (imt/imt.max(1, keepdim=True)[0] > self.threshold).float()
				  # Use large negative number to mask actions from argmax
				  return ((imt * q + (1. - imt) * -1e8).argmax(1))  #int((imt * q + (1. - imt) * -1e8).argmax(1))
					
		  else:
			  return np.random.randint(self.num_actions)
   
  def train_epoch(self,data_loader,it=0):
    sum_q_loss=0
    sum_i_loss=0
    for batch,(state,next_state,action,reward,done) in enumerate(data_loader):
       # Compute the target Q value
      
      with torch.no_grad():
          batch_size=state.shape[0]
          action=torch.LongTensor(action.reshape(batch_size,1)).to(device)
          reward=torch.FloatTensor(reward.reshape(batch_size,1).cpu().numpy()).to(device)
          done=done.reshape(batch_size,1).to(device)
          
          
          q, imt, i = self.Q(next_state)
          imt = imt.exp()
          imt = (imt/imt.max(1, keepdim=True)[0] > self.threshold).float()

          # Use large negative number to mask actions from argmax
          next_action = (imt * q + (1 - imt) * -1e8).argmax(1, keepdim=True)

          q, imt, i = self.Q_target(next_state)
          target_Q = reward + done * self.discount * q.gather(1, next_action).reshape(-1, 1)

      current_Q, imt, i = self.Q(state)
      current_Q = current_Q.gather(1, action)

      # Compute Q loss
      q_loss = F.smooth_l1_loss(current_Q, target_Q)
      i_loss = F.nll_loss(imt, action.reshape(-1))

      Q_loss = q_loss + i_loss + 1e-2 * i.pow(2).mean()
      sum_q_loss+=q_loss.item()
      sum_i_loss+=i_loss.item()
	 
      if batch%25==0:
        print('Epoch :',it,'Batch :', batch,'Average q Loss :',sum_q_loss/(batch+1))
        print('Epoch :',it,'Batch :', batch,'Average i Loss :',sum_i_loss/(batch+1))


      # Optimize the Q
      self.Q_optimizer.zero_grad()
      Q_loss.backward()
      self.Q_optimizer.step()

		  # Update target network by polyak or full copy every X iterations.
      self.iterations += 1
      self.maybe_update_target()

  def polyak_target_update(self):
      for param, target_param in zip(self.Q.parameters(), self.Q_target.parameters()):
        target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
		    


  def copy_target_update(self):
        if self.iterations % self.target_update_frequency == 0:
          self.Q_target.load_state_dict(self.Q.state_dict())
			    

In [ ]:
dBCQ=discrete_BCQ(
    is_atari=False,
		num_actions=25,
		state_dim=41,
		device=device,
		BCQ_threshold=0.3,
		discount=0.99,
		optimizer="Adam",
		optimizer_parameters={},
		polyak_target_update=True,
		target_update_frequency=8e3,
		tau=0.005,
		initial_eps = 1,
		end_eps = 0.001,
		eps_decay_period = 25e4,
		eval_eps=0,
	)

In [ ]:
cd dBCQ/

/content/drive/My Drive/sepsis3-cohort/dBCQ


In [ ]:
ls

dBCQ_0.pt   dBCQ_13.pt  dBCQ__20.pt  dBCQ_4.pt  dBCQ_8.pt
dBCQ_10.pt  dBCQ_15.pt  dBCQ__21.pt  dBCQ_5.pt  dBCQ_9.pt
dBCQ_11.pt  dBCQ_16.pt  dBCQ_2.pt    dBCQ_6.pt  dBCQ_w_1.pt
dBCQ_12.pt  dBCQ_1.pt   dBCQ_3.pt    dBCQ_7.pt


In [ ]:
checkpoint=torch.load('dBCQ_16.pt', map_location=torch.device('cpu'))
dBCQ.Q.load_state_dict(checkpoint['dBCQ_state_dict'])
dBCQ.Q_target.load_state_dict(checkpoint['dBCQ_target_state_dict'])

<All keys matched successfully>

In [ ]:
for i in range(1,1000):
  dBCQ.train_epoch(loader,i)
  torch.save({
            'dBCQ_state_dict': dBCQ.Q.state_dict(),
             'dBCQ_target_state_dict': dBCQ.Q_target.state_dict(),
                
            }, 'dBCQ_{}.pt'.format(i))

##### Weighted Loader

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,Vaso_cuts,Fluid_cuts,action
0,-1.046870,1.974396,-1.110013,0.969004,-1.378317,-0.991096,-0.647660,-0.646901,0.0,0.0,0.0,-0.777651,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.502859,1.030435,0.437960,65.957603,1.096018,0.0,-0.308252,0.0,0.0,-0.250,0,0,0
1,-1.394644,1.544799,-1.367532,0.441438,-0.828779,-0.656335,-0.647660,-0.646901,1.0,0.0,0.0,-0.080965,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.482850,0.995847,0.452339,66.413605,1.096018,0.0,-0.308252,0.0,0.0,-0.375,0,0,0
2,-1.185979,1.360686,-1.110013,-0.261983,-0.004472,0.347947,-0.230333,-0.646901,1.0,0.0,0.0,2.009094,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.521583,0.918802,0.458562,66.626884,1.096018,0.0,-0.308252,0.0,0.0,-0.025,0,0,0
3,-1.394644,0.624235,-1.238772,-0.086127,-0.004472,-0.321574,-0.230333,-0.646901,1.0,0.0,0.0,2.009094,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.565697,0.881036,0.521476,67.943565,1.096018,0.0,-0.308252,0.0,0.0,-0.025,0,0,0
4,-1.046870,0.562864,-0.916875,-0.086127,-0.004472,-1.325856,0.047884,-0.646901,1.0,0.0,0.0,2.009094,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.604537,0.859168,0.516133,67.920723,1.096018,0.0,-0.308252,0.0,0.0,-0.025,0,0,0


In [ ]:
weights=df.Vaso_cuts+df.Fluid_cuts+0.5
weights=weights.values
sampler = torch.utils.data.sampler.WeightedRandomSampler(torch.DoubleTensor(weights),num_samples=len(weights))
new_loader =DataLoader(ds, batch_size=100,sampler=sampler)

In [ ]:
dBCQ.Q_optimizer.lr=1e-5

In [ ]:
for i in range(1,1000):
   dBCQ.train_epoch(new_loader,i)
   torch.save({
            'dBCQ_state_dict': dBCQ.Q.state_dict(),
             'dBCQ_target_state_dict': dBCQ.Q_target.state_dict(),
                
            }, 'dBCQ_w_{}.pt'.format(i))


### Action Selection

In [45]:
def get_actions(model):
  test_loader=DataLoader(ds,batch_size=256,shuffle=False)
  actions=[]
  for batch,(state,next_state,action,reward,done) in enumerate(test_loader):
    action=model.select_action(state)
    print(100*(batch+1)/(len(test_loader)),' % Done')
    actions.append(action.cpu().numpy())
    return actions

In [ ]:
dBCQ_actions=np.hstack(actions)

In [ ]:
df['dBCQ']=dBCQ_actions

In [ ]:
df['dBCQ'].value_counts(normalize=True)

0     0.762639
4     0.065889
23    0.059713
3     0.048758
1     0.037147
2     0.025808
19    0.000046
Name: dBCQ, dtype: float64

In [ ]:
# df['dBCQ'].to_csv('dBCQ.csv')

In [ ]:
cd dBCQ/

/content/drive/My Drive/sepsis3-cohort/dBCQ


In [ ]:
dBCQ=pd.read_csv('dBCQ.csv')

In [ ]:
dBCQ=dBCQ.iloc[:,1]

In [ ]:
dBCQ.value_counts(normalize=True)

0     0.762639
4     0.065889
23    0.059713
3     0.048758
1     0.037147
2     0.025808
19    0.000046
Name: dBCQ, dtype: float64

In [ ]:
acts=[]
for i,val in enumerate(dBCQ.values):
  print(i*100/df.shape[0])
  acts.append(np.argwhere(all_acts==val))


In [ ]:
acts__=np.vstack(acts)

In [ ]:
# dBCQ_.to_csv('dBCQ.csv',index=False)

In [ ]:
dBCQ_=pd.DataFrame(acts__)

In [ ]:
dBCQ_.value_counts(normalize=True)

Vaso  Fluids
0     0         0.762639
      4         0.065889
4     3         0.059713
0     3         0.048758
      1         0.037147
      2         0.025808
3     4         0.000046
dtype: float64

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,Vaso_cuts,Fluid_cuts,action
0,46.0,118.0,62.0,26.0,0.0,94.0,109.0,36.388889,0.0,0.0,0.0,0.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.502859,1.030435,0.437960,65.957603,82.0,0.0,75.1,0.0,0.0,-0.250,0,0,0
1,41.0,111.0,58.0,23.0,2.0,95.0,109.0,36.388889,1.0,0.0,0.0,1.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462475,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.482850,0.995847,0.452339,66.413605,82.0,0.0,75.1,0.0,0.0,-0.375,0,0,0
2,44.0,108.0,62.0,19.0,5.0,98.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.521583,0.918802,0.458562,66.626884,82.0,0.0,75.1,0.0,0.0,-0.025,0,0,0
3,41.0,96.0,60.0,20.0,5.0,96.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.565697,0.881036,0.521476,67.943565,82.0,0.0,75.1,0.0,0.0,-0.025,0,0,0
4,46.0,95.0,65.0,20.0,5.0,93.0,124.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266840,0.538169,0.202432,-0.357259,-0.794549,0.604537,0.859168,0.516133,67.920723,82.0,0.0,75.1,0.0,0.0,-0.025,0,0,0


In [ ]:
dBCQ_['Clinician_Vaso_cuts']=df['Vaso_cuts']
dBCQ_['Clinician_Fluids_cuts']=df['Fluid_cuts']



In [ ]:
dBCQ_.head()

,Vaso,Fluids,Clinician_Vaso,Clinician_Fluids,Clinician_Vaso_cuts,Clinician_Fluids_cuts
0,0,0,0.0,0.0,0,0
1,0,0,0.0,0.0,0,0
2,0,0,0.0,0.0,0,0
3,0,0,0.0,0.0,0,0
4,0,0,0.0,0.0,0,0


In [ ]:
dBCQ_[['Vaso','Clinician_Vaso_cuts']].value_counts(normalize=True)

Vaso  Clinician_Vaso_cuts
0     0                      0.896952
4     4                      0.017019
0     1                      0.015444
4     3                      0.013740
0     2                      0.011909
4     2                      0.010920
0     3                      0.009589
4     0                      0.009583
      1                      0.008452
0     4                      0.006346
3     3                      0.000028
      0                      0.000007
      2                      0.000005
      4                      0.000003
      1                      0.000003
dtype: float64

In [ ]:
dBCQ_[['Fluids','Clinician_Fluids_cuts']].value_counts(normalize=True)

Fluids  Clinician_Fluids_cuts
0       0                        0.557811
        1                        0.070571
4       4                        0.064258
0       2                        0.059357
3       3                        0.053904
0       3                        0.043470
        4                        0.031430
1       1                        0.030727
2       2                        0.023781
3       0                        0.019632
        2                        0.015247
        1                        0.011239
        4                        0.008449
1       2                        0.003502
        0                        0.002247
4       0                        0.001189
2       0                        0.000960
        1                        0.000558
1       3                        0.000408
2       3                        0.000370
1       4                        0.000264
4       2                        0.000208
        1                        0.000182
2   

## DDQN

In [23]:
dBCQ=discrete_BCQ(
    is_atari=False,
		num_actions=25,
		state_dim=41,
		device=device,
		BCQ_threshold=0.008,
		discount=0.99,
		optimizer="Adam",
		optimizer_parameters={},
		polyak_target_update=True,
		target_update_frequency=8e3,
		tau=0.005,
		initial_eps = 1,
		end_eps = 0.001,
		eps_decay_period = 25e4,
		eval_eps=0,
	)

In [25]:
ls

dBCQ_0.pt   dBCQ_13.pt  dBCQ__20.pt  dBCQ_4.pt  dBCQ_8.pt    ddqn.pt
dBCQ_10.pt  dBCQ_15.pt  dBCQ__21.pt  dBCQ_5.pt  dBCQ_9.pt
dBCQ_11.pt  dBCQ_16.pt  dBCQ_2.pt    dBCQ_6.pt  dBCQ.csv
dBCQ_12.pt  dBCQ_1.pt   dBCQ_3.pt    dBCQ_7.pt  dBCQ_w_1.pt


In [26]:
checkpoint=torch.load('ddqn.pt', map_location=torch.device('cpu'))
dBCQ.Q.load_state_dict(checkpoint['dBCQ_state_dict'])
dBCQ.Q_target.load_state_dict(checkpoint['dBCQ_target_state_dict'])

<All keys matched successfully>

In [ ]:
for i in range(1,1000):
  dBCQ.train_epoch(loader,i)
  torch.save({
            'dBCQ_state_dict': dBCQ.Q.state_dict(),
             'dBCQ_target_state_dict': dBCQ.Q_target.state_dict(),
                
            }, 'ddqn.pt'.format(i))

In [ ]:
actions=get_actions(dBCQ)
ddQN_actions=np.hstack(actions)
print(ddQN_actions.shape)

In [ ]:
acts=[]
for i,val in enumerate(ddQN_actions):
  print(i*100/df.shape[0])
  acts.append(np.argwhere(all_acts==val))
actions_=np.vstack(acts)
assert actions_.shape[1]==2

In [46]:
dBCQ.head()

,Vaso,Fluids,Clinician_Vaso,Clinician_Fluids,Clinician_Vaso_cuts,Clinician_Fluids_cuts,DDQN_Vaso,DDQN_Fluids
0,0,0,0.0,0.0,0,0,0,0
1,0,0,0.0,0.0,0,0,0,0
2,0,0,0.0,0.0,0,0,0,0
3,0,0,0.0,0.0,0,0,0,0
4,0,0,0.0,0.0,0,0,0,0


In [39]:
dBCQ['DDQN_Vaso']=actions_[:,0]
dBCQ['DDQN_Fluids']=actions_[:,1]

In [41]:
dBCQ[['DDQN_Vaso','DDQN_Fluids']].value_counts(normalize=True)

DDQN_Vaso  DDQN_Fluids
2          3              0.558070
0          0              0.403433
           4              0.027655
           2              0.010842
dtype: float64

In [43]:
dBCQ.to_csv('All_actions.csv')
print('done')

done


In [50]:
dBCQ[['Clinician_Fluids_cuts','DDQN_Fluids']].value_counts(normalize=True)

Clinician_Fluids_cuts  DDQN_Fluids
0                      0              0.340140
                       3              0.241587
1                      3              0.106546
3                      3              0.094330
2                      3              0.079587
4                      0              0.040184
                       3              0.036021
                       4              0.027572
2                      0              0.012490
                       2              0.010015
1                      0              0.006708
3                      0              0.003910
4                      2              0.000763
0                      4              0.000073
                       2              0.000038
1                      2              0.000020
3                      2              0.000006
1                      4              0.000004
3                      4              0.000003
2                      4              0.000003
dtype: float64